In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# FE'li dataset
df = pd.read_csv("../data/processed/train_fe.csv")
print("FE dataset shape:", df.shape)

# Hedef ve feature
y = df["TARGET"]
X = df.drop(columns=["TARGET"])

# Kategorik kolonlar
cat_cols = X.select_dtypes(include=["object"]).columns
le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    le_dict[col] = le

# Aynı split mantığı: (random_state, test_size, stratify aynı olursa öncekiyle tutarlı olur)
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train:", X_train.shape, "Val:", X_val.shape)

# Final modeli yükle (04. notebook'ta kaydetmiştik)
final_model = joblib.load("../models/final_model.pkl")
print("Model loaded.")


FE dataset shape: (307511, 134)
Train: (246008, 133) Val: (61503, 133)
Model loaded.
